# Maguy Abou Ghosn Voice Cloning Interface using RVC

## 1. Setup & Installation

In [ ]:
# Clone the Advanced RVC Inference repository
!git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
%cd Advanced-RVC-Inference

Cloning into 'Advanced-RVC-Inference'...
remote: Enumerating objects: 1365, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 1365 (delta 117), reused 70 (delta 66), pack-reused 1206 (from 2)
Receiving objects: 100% (1365/1365), 5.12 MiB | 10.80 MiB/s, done.
Resolving deltas: 100% (631/631), done.
/content/Advanced-RVC-Inference


In [ ]:
# Install required dependencies
!pip install -r requirements.txt
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install numpy==1.23.5

Ignoring pip: markers 'sys_platform == "darwin"' don't match your environment
Ignoring wheel: markers 'sys_platform == "darwin"' don't match your environment
Ignoring PyYAML: markers 'sys_platform == "darwin"' don't match your environment
Ignoring omegaconf: markers 'sys_platform == "darwin"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
# Add our current directory to system path
import os
import sys
%cd Advanced-RVC-Inference
sys.path.append('/content/Advanced-RVC-Inference')

# Download prerequisites
!python programs/applio_code/rvc/lib/tools/prerequisites_download.py

/content/Advanced-RVC-Inference


In [ ]:
#@title ## Load model files(.pth & .index) from Google Drive or upload them
load_from_drive = True  # @param {type:"boolean"}

if load_from_drive:
    from google.colab import drive
    drive.mount('/content/drive')

    # Change these to your actual file locations
    drive_model_path = "/content/drive/MyDrive/maguy_rvc_model/maguymodel.pth"  # @param {type:"string"}
    drive_index_path = "/content/drive/MyDrive/maguy_rvc_model/added_IVF1772_Flat_nprobe_1_maguymodel_v2.index"  # @param {type:"string"}

    !mkdir -p logs/custom_model
    custom_model_path = f"logs/custom_model/{os.path.basename(drive_model_path)}"
    custom_index_path = f"logs/custom_model/{os.path.basename(drive_index_path)}"

    !cp "{drive_model_path}" "{custom_model_path}"
    !cp "{drive_index_path}" "{custom_index_path}"

else:
    from google.colab import files

    print("Please upload your .pth model file...")
    uploaded_model = files.upload()
    model_filename = list(uploaded_model.keys())[0]
    custom_model_path = f"logs/custom_model/{model_filename}"
    !mkdir -p logs/custom_model
    !mv "{model_filename}" "{custom_model_path}"

    print("Please upload your .index file...")
    uploaded_index = files.upload()
    index_filename = list(uploaded_index.keys())[0]
    custom_index_path = f"logs/custom_model/{index_filename}"
    !mv "{index_filename}" "{custom_index_path}"

print(f"✅ Model path: {custom_model_path}")
print(f"✅ Index path: {custom_index_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model path: logs/custom_model/maguymodel.pth
✅ Index path: logs/custom_model/added_IVF1772_Flat_nprobe_1_maguymodel_v2.index


## 2. VoiceConverter Class

In [ ]:
class VoiceConverter:
    """
    A simple wrapper class for Advanced RVC Inference that handles voice conversion
    with minimal setup.
    """

    def __init__(self, model_path: str, index_path: str):
        """
        Initialize the voice converter with model and index files.

        Args:
            model_path: Path to the .pth model file
            index_path: Path to the .index file
        """
        self.model_path = model_path
        self.index_path = index_path
        self.converter = None

        # Import the necessary modules from the RVC project
        repo_dir = '/content/Advanced-RVC-Inference'  # Update with the correct path
        sys.path.append(repo_dir)

        # Load the voice converter
        from programs.applio_code.rvc.infer.infer import VoiceConverter
        self.converter = VoiceConverter()

    def convert(self, audio_path: str, output_path: str = None):
        """
        Convert an audio file using the loaded voice model.

        Args:
            audio_path: Path to the input audio file
            output_path: Optional path to save the output audio file. If not provided,
                        a default path will be used based on the input filename.

        Returns:
            Tuple containing (message, output_file_path)
        """
        if self.converter is None:
            raise RuntimeError("Converter not initialized properly")

        # Set default output path if not provided
        if output_path is None:
            dir_name = os.path.dirname(audio_path)
            base_name = os.path.basename(audio_path)
            name_without_ext = os.path.splitext(base_name)[0]
            output_path = os.path.join(dir_name, f"{name_without_ext}_converted.wav")

        # Default settings
        embedder_model = "contentvec"
        pitch = 0
        pitch_method = "rmvpe"
        index_rate = 0.75
        volume_envelope = 0.25
        protect = 0.33
        hop_length = 64
        split_audio = False
        autotune = False
        filter_radius = 3

        # Perform the conversion
        self.converter.convert_audio(
            audio_input_path=audio_path,
            audio_output_path=output_path,
            model_path=self.model_path,
            index_path=self.index_path,
            embedder_model=embedder_model,
            pitch=pitch,
            f0_file=None,
            f0_method=pitch_method,
            index_rate=index_rate,
            volume_envelope=volume_envelope,
            protect=protect,
            hop_length=hop_length,
            split_audio=split_audio,
            f0_autotune=autotune,
            filter_radius=filter_radius,
            embedder_model_custom=None,
            export_format="WAV"
        )

        return "Conversion completed successfully", output_path

## 3. Gradio Interface

In [ ]:
import gradio as gr
import tempfile
import uuid

# Model and index path to use
model_path = "logs/custom_model/maguymodel.pth"
index_path = "logs/custom_model/added_IVF1772_Flat_nprobe_1_maguymodel_v2.index"


# Initialize your converter as before
converter = VoiceConverter(custom_model_path, custom_index_path)

def process_and_convert(audio_filepath):
    # audio_filepath is already a .wav file path recorded by Gradio
    output_path = tempfile.NamedTemporaryFile(suffix="_converted.wav", delete=False).name
    _, result_path = converter.convert(audio_filepath, output_path=output_path)
    return result_path

# Build the Blocks app
with gr.Blocks(title="🎤 Maguy Abou Ghosn Voice Cloner", theme="default") as demo:
    gr.HTML(
        f"""
        <h1 style='text-align: center;'> 🎤 Maguy Abou Ghosn Voice Cloner </h1>
        <p style='text-align: center;'> Record your voice and get an instant voice‑converted clip.</a>
        """
    )

    with gr.Row():
        input_audio = gr.Audio(sources=["microphone","upload"], type="filepath", label="🎙️ Record Your Voice")
        output_audio = gr.Audio(type="filepath", label="🗣️ Cloned Voice", autoplay = True)

    convert_btn = gr.Button("Convert Voice", variant="primary")

    # Wire up the button
    input_audio.stop_recording(
        fn=process_and_convert,
        inputs=input_audio,
        outputs=output_audio
    )
    convert_btn.click(
        fn=process_and_convert,
        inputs=input_audio,
        outputs=output_audio
    )

# Launch with a public link
demo.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://06e8f71690b8c046aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Converting audio '/tmp/gradio/ebfa8a05e2537d207261c99c94923556faf69fbd988d9dbbb1ac9b09fe46b19c/audio3.wav'...
Conversion completed at '/tmp/tmp1a7f1t6x_converted.wav' in 3.27 seconds.
Converting audio '/tmp/gradio/c1a429a528eb40ba25a2ea0c81c84f346add8d6e60de444324626ca3fd23afb1/audio5.wav'...
Conversion completed at '/tmp/tmphbv943wr_converted.wav' in 2.00 seconds.
Converting audio '/tmp/gradio/6ede30e9453c657e4a7f4cc4c283b8eb2f796f48818d8432451096e0d84b496f/audio.wav'...
Conversion completed at '/tmp/tmpm4_acvwd_converted.wav' in 3.05 seconds.
Converting audio '/tmp/gradio/d69edbc9a057f53f1a054481a24ff87f96e82740c9da04d85b5c56fd56ea7656/audio4.wav'...
Conversion completed at '/tmp/tmp_8a8tc9x_converted.wav' in 2.25 seconds.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://d78cd53745d4f2d22a.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://06e8f71690b8c046aa.gradio.live
